In [6]:
import json 
import numpy as np
import faiss
import os

# Path file
embd_path = r"D:/chatbot_covid19_intern_procodecg/vector_db/faiss__metadata.json"
faiss_path = r"D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index"

# Pastikan folder tujuan ada
os.makedirs(os.path.dirname(faiss_path), exist_ok=True)

# List untuk menampung data
text = []
vectors = []

# Membaca file embedding
with open(embd_path, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        text.append(record["text"])
        vectors.append(record["embedding"])

# Ubah list ke numpy array
vectors = np.array(vectors).astype("float32")

# Buat FAISS index (L2 distance)
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

# Simpan FAISS index ke file .index
faiss.write_index(index, faiss_path)

# Info hasil
print(f"💚 FAISS index disimpan di : {faiss_path}")
print(f"Total data: {len(text)} chunk")

# Simpan teks referensi dalam file JSON (bukan FAISS)
text_json_path = faiss_path.replace(".index", "_texts.json")
with open(text_json_path, "w", encoding="utf-8") as f:
    json.dump(text, f, ensure_ascii=False, indent=2)

print(f"📝 File teks disimpan di : {text_json_path}")


💚 FAISS index disimpan di : D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index
Total data: 20 chunk
📝 File teks disimpan di : D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19_texts.json


In [7]:
import os

faiss_path = "D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index"
print("Ada file?:", os.path.exists(faiss_path))
print("Ukuran file (byte):", os.path.getsize(faiss_path) if os.path.exists(faiss_path) else 0)


Ada file?: True
Ukuran file (byte): 61485


In [3]:
with open("D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index", "rb") as f:
    header = f.read(20)
print(header)


b'IxF2\x00\x03\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00'


In [5]:
import faiss

# --- 1. Muat (Load) Indeks FAISS ---
# Ganti path sesuai dengan lokasi file Anda jika perlu
index = faiss.read_index("D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index")

# --- 2. Cek Dimensi Vektor (d) ---
dimensi_faiss = index.d

print(f"✅ Dimensi Vektor (index.d) dalam file FAISS: {dimensi_faiss}")

✅ Dimensi Vektor (index.d) dalam file FAISS: 768


In [1]:
# create_faiss_index.py
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import time

def setup_paths():
    """Setup project paths"""
    project_root = r"D:/chatbot_covid19_intern_procodecg"
    faiss_dir = os.path.join(project_root, "faiss")
    data_dir = os.path.join(project_root, "data")
    
    # Buat folder jika belum ada
    os.makedirs(faiss_dir, exist_ok=True)
    os.makedirs(data_dir, exist_ok=True)
    
    return {
        'project_root': project_root,
        'faiss_dir': faiss_dir,
        'data_dir': data_dir,
        'index_path': os.path.join(faiss_dir, "faiss_textcovid19.index"),
        'texts_path': os.path.join(faiss_dir, "faiss_textcovid19_texts.json"),
        'chunks_path': os.path.join(data_dir, "chunks.json"),
        'jsonl_path': r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
    }

def load_and_process_jsonl(jsonl_path):
    """Load dan process file JSONL"""
    print(f"📖 Loading JSONL from: {jsonl_path}")
    
    if not os.path.exists(jsonl_path):
        print(f"❌ JSONL file not found: {jsonl_path}")
        return []
    
    chunks = []
    try:
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                line = line.strip()
                if line:
                    try:
                        data = json.loads(line)
                        
                        # Extract text dari berbagai kemungkinan field
                        text = ""
                        if 'text' in data:
                            text = data['text']
                        elif 'content' in data:
                            text = data['content'] 
                        elif 'chunk' in data:
                            text = data['chunk']
                        elif 'page_content' in data:
                            text = data['page_content']
                        else:
                            # Coba cari field yang mengandung text
                            for key, value in data.items():
                                if isinstance(value, str) and len(value) > 20:
                                    text = value
                                    break
                            if not text:
                                text = str(data)
                        
                        # Clean text
                        if text:
                            text = text.strip()
                            if len(text) > 10:  # Minimal 10 karakter
                                chunks.append({
                                    'id': len(chunks),
                                    'text': text,
                                    'source': data.get('source', 'unknown'),
                                    'metadata': data.get('metadata', {})
                                })
                        
                        # Progress indicator
                        if (i + 1) % 100 == 0:
                            print(f"   Processed {i + 1} lines...")
                            
                    except json.JSONDecodeError as e:
                        print(f"⚠️ JSON error line {i}: {e}")
                        continue
                    except Exception as e:
                        print(f"⚠️ Error processing line {i}: {e}")
                        continue
        
        print(f"✅ Successfully loaded {len(chunks)} chunks from JSONL")
        return chunks
        
    except Exception as e:
        print(f"❌ Error loading JSONL: {e}")
        import traceback
        traceback.print_exc()
        return []

def create_embeddings(chunks):
    """Create embeddings untuk semua chunks"""
    print("🔄 Loading embedding model...")
    
    try:
        model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        
        print("🔄 Creating embeddings...")
        texts = [chunk['text'] for chunk in chunks]
        
        # Create embeddings dengan batch processing
        batch_size = 32
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False)
            all_embeddings.append(batch_embeddings)
            
            # Progress
            if (i // batch_size) % 10 == 0:
                print(f"   Processed {min(i + batch_size, len(texts))}/{len(texts)} chunks...")
        
        # Combine all embeddings
        embeddings = np.vstack(all_embeddings).astype('float32')
        
        print(f"✅ Created embeddings: {embeddings.shape}")
        return embeddings, model
        
    except Exception as e:
        print(f"❌ Error creating embeddings: {e}")
        raise e

def create_faiss_index(embeddings):
    """Create FAISS index dari embeddings"""
    print("🔄 Creating FAISS index...")
    
    try:
        dimension = embeddings.shape[1]
        
        # Gunakan IndexFlatIP untuk cosine similarity
        index = faiss.IndexFlatIP(dimension)
        
        # Normalize vectors untuk cosine similarity
        faiss.normalize_L2(embeddings)
        
        # Add ke index
        index.add(embeddings)
        
        print(f"✅ FAISS index created with {index.ntotal} vectors")
        return index
        
    except Exception as e:
        print(f"❌ Error creating FAISS index: {e}")
        raise e

def save_index_data(chunks, index, paths):
    """Save index dan data terkait"""
    print("💾 Saving index and data...")
    
    try:
        # Save chunks dengan metadata
        with open(paths['chunks_path'], 'w', encoding='utf-8') as f:
            json.dump(chunks, f, ensure_ascii=False, indent=2)
        print(f"✅ Saved chunks to: {paths['chunks_path']}")
        
        # Save texts untuk retriever (hanya teks)
        texts_for_retriever = [chunk['text'] for chunk in chunks]
        with open(paths['texts_path'], 'w', encoding='utf-8') as f:
            json.dump(texts_for_retriever, f, ensure_ascii=False, indent=2)
        print(f"✅ Saved texts to: {paths['texts_path']}")
        
        # Save FAISS index
        faiss.write_index(index, paths['index_path'])
        print(f"✅ Saved FAISS index to: {paths['index_path']}")
        
    except Exception as e:
        print(f"❌ Error saving data: {e}")
        raise e

def verify_index(paths):
    """Verify bahwa index berhasil dibuat"""
    print("\n🔍 Verifying index...")
    
    try:
        # Check files exist
        required_files = {
            'FAISS Index': paths['index_path'],
            'Texts File': paths['texts_path'],
            'Chunks File': paths['chunks_path']
        }
        
        for name, path in required_files.items():
            exists = os.path.exists(path)
            size = os.path.getsize(path) if exists else 0
            print(f"   {name}: {'✅' if exists else '❌'} {path} ({size} bytes)")
        
        # Test load index
        index = faiss.read_index(paths['index_path'])
        print(f"   Index vectors: ✅ {index.ntotal}")
        
        # Test load texts
        with open(paths['texts_path'], 'r', encoding='utf-8') as f:
            texts = json.load(f)
        print(f"   Text chunks: ✅ {len(texts)}")
        
        return True
        
    except Exception as e:
        print(f"❌ Verification failed: {e}")
        return False

def test_search(paths, test_queries=None):
    """Test search functionality"""
    if test_queries is None:
        test_queries = [
            "Siapa orang pertama yang divaksin di Indonesia?",
            "Apa vaksin kedua yang digunakan?",
            "Jenis vaksin COVID-19 di Indonesia",
            "Presiden Jokowi vaksin",
            "Gejala COVID-19",
            "Aplikasi PeduliLindungi"
        ]
    
    print(f"\n🔍 Testing search with {len(test_queries)} queries...")
    
    try:
        # Load model
        model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        
        # Load index dan texts
        index = faiss.read_index(paths['index_path'])
        with open(paths['texts_path'], 'r', encoding='utf-8') as f:
            texts = json.load(f)
        
        results_summary = []
        
        for query in test_queries:
            print(f"\n📝 Query: '{query}'")
            
            # Encode query
            query_embedding = model.encode([query], convert_to_numpy=True).astype("float32")
            faiss.normalize_L2(query_embedding)
            
            # Search
            scores, indices = index.search(query_embedding, 3)
            
            found_good = False
            for i, (idx, score) in enumerate(zip(indices[0], scores[0])):
                if 0 <= idx < len(texts):
                    quality = "✅ GOOD" if score > 0.3 else "⚠️ WEAK" if score > 0.15 else "❌ POOR"
                    print(f"   {i+1}. Score: {score:.3f} [{quality}]")
                    
                    if score > 0.3:
                        found_good = True
                        text_preview = texts[idx][:100] + "..." if len(texts[idx]) > 100 else texts[idx]
                        print(f"       Text: {text_preview}")
            
            results_summary.append({
                'query': query,
                'found_good_match': found_good,
                'top_score': scores[0][0] if len(scores[0]) > 0 else 0
            })
            
            if not found_good:
                print("   ❌ NO GOOD MATCHES FOUND!")
        
        # Summary
        good_matches = sum(1 for r in results_summary if r['found_good_match'])
        print(f"\n📊 SEARCH TEST SUMMARY:")
        print(f"   Good matches: {good_matches}/{len(test_queries)}")
        print(f"   Success rate: {(good_matches/len(test_queries))*100:.1f}%")
        
        return good_matches > 0
        
    except Exception as e:
        print(f"❌ Search test failed: {e}")
        return False

def main():
    """Main function untuk create FAISS index"""
    print("🚀 STARTING FAISS INDEX CREATION...")
    start_time = time.time()
    
    try:
        # Setup paths
        paths = setup_paths()
        print(f"📁 Project root: {paths['project_root']}")
        print(f"📁 FAISS dir: {paths['faiss_dir']}")
        print(f"📁 JSONL source: {paths['jsonl_path']}")
        
        # Step 1: Load data dari JSONL
        print("\n" + "="*50)
        print("📖 STEP 1: Loading data from JSONL...")
        chunks = load_and_process_jsonl(paths['jsonl_path'])
        
        if not chunks:
            print("❌ No chunks loaded! Exiting.")
            return False
        
        # Step 2: Create embeddings
        print("\n" + "="*50)
        print("🔤 STEP 2: Creating embeddings...")
        embeddings, model = create_embeddings(chunks)
        
        # Step 3: Create FAISS index
        print("\n" + "="*50)
        print("📊 STEP 3: Creating FAISS index...")
        index = create_faiss_index(embeddings)
        
        # Step 4: Save data
        print("\n" + "="*50)
        print("💾 STEP 4: Saving index and data...")
        save_index_data(chunks, index, paths)
        
        # Step 5: Verify
        print("\n" + "="*50)
        print("✅ STEP 5: Verification...")
        verification_ok = verify_index(paths)
        
        # Step 6: Test search
        print("\n" + "="*50)
        print("🔍 STEP 6: Testing search...")
        search_ok = test_search(paths)
        
        # Final summary
        print("\n" + "="*50)
        print("🎉 FAISS INDEX CREATION COMPLETED!")
        print(f"📊 Total chunks: {len(chunks)}")
        print(f"📊 Embedding dimension: {embeddings.shape[1]}")
        print(f"📊 FAISS index size: {index.ntotal} vectors")
        print(f"⏱️ Total time: {time.time() - start_time:.2f} seconds")
        
        if verification_ok and search_ok:
            print("\n✅ SUCCESS: Index created and tested successfully!")
            print("\n📋 NEXT STEPS:")
            print("1. Update your retriever.py to use the new index")
            print("2. Clear Streamlit cache")
            print("3. Test your chatbot!")
        else:
            print("\n⚠️ WARNING: Index created but some tests failed")
            
        return True
        
    except Exception as e:
        print(f"\n❌ FAILED: {e}")
        import traceback
        traceback.print_exc()
        return False

if __name__ == "__main__":
    success = main()
    if success:
        print("\n🎉 Script completed successfully!")
    else:
        print("\n💥 Script failed!")

🚀 STARTING FAISS INDEX CREATION...
📁 Project root: D:/chatbot_covid19_intern_procodecg
📁 FAISS dir: D:/chatbot_covid19_intern_procodecg\faiss
📁 JSONL source: D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl

📖 STEP 1: Loading data from JSONL...
📖 Loading JSONL from: D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl
✅ Successfully loaded 20 chunks from JSONL

🔤 STEP 2: Creating embeddings...
🔄 Loading embedding model...
🔄 Creating embeddings...
   Processed 20/20 chunks...
✅ Created embeddings: (20, 768)

📊 STEP 3: Creating FAISS index...
🔄 Creating FAISS index...
✅ FAISS index created with 20 vectors

💾 STEP 4: Saving index and data...
💾 Saving index and data...
✅ Saved chunks to: D:/chatbot_covid19_intern_procodecg\data\chunks.json
✅ Saved texts to: D:/chatbot_covid19_intern_procodecg\faiss\faiss_textcovid19_texts.json
✅ Saved FAISS index to: D:/chatbot_covid19_intern_procodecg\faiss\faiss_textcovid19.index

✅ STEP